## Step1: Import Library

In [3]:

!pip install --upgrade nbformat
!pip install nltk
!pip install spacy # spaCy is an open-source software library for advanced natural language processing
!pip install WordCloud
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing
import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
# import keras
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 
# setting the style of the notebook to be monokai theme  
# this line of code is important to ensure that we are able to see the x and y axes clearly
# If you don't run this code line, you will notice that the xlabel and ylabel on any plot is black on black and it will be hard to see them. 


[nltk_data] Downloading package punkt to /Users/magzhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'seaborn'

## Step2: Import and Clean Data

In [4]:
df = pd.read_csv("/Users/magzhang/Downloads/train.csv")

In [5]:
#new_row1=['Donald Trump won 2020 election','0']
#new_row2=['Donald Trump won 2016 election','1']
#df.loc[-2]=new_row1
#df.loc[-1]=new_row2
#df.reset_index(drop=True,inplace=True)

In [6]:
df

,Statement,Label
0,Says the Annies List political group supports ...,False
1,When did the decline of coal start? It started...,True
2,"Hillary Clinton agrees with John McCain ""by vo...",True
3,Health care reform legislation is likely to ma...,False
4,The economic turnaround started at the end of ...,True
...,...,...
10235,There are a larger number of shark attacks in ...,True
10236,Democrats have now become the party of the [At...,True
10237,Says an alternative to Social Security that op...,True
10238,On lifting the U.S. Cuban embargo and allowing...,False


In [423]:
df['label'] = df['Label'].astype(int)

In [424]:
df.drop(["Label"],  axis=1, inplace=True)

In [425]:
df

,Statement,label
0,Says the Annies List political group supports ...,0
1,When did the decline of coal start? It started...,1
2,"Hillary Clinton agrees with John McCain ""by vo...",1
3,Health care reform legislation is likely to ma...,0
4,The economic turnaround started at the end of ...,1
...,...,...
10235,There are a larger number of shark attacks in ...,1
10236,Democrats have now become the party of the [At...,1
10237,Says an alternative to Social Security that op...,1
10238,On lifting the U.S. Cuban embargo and allowing...,0


In [426]:
df['Statement'][8]

'However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations Center-Pacific.'

In [427]:
df.groupby(['label']).count()

,Statement
label,
0,4488
1,5752


 ## Step4 PERFORM DATA CLEANING

In [360]:
# download stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/magzhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [428]:
# Obtain additional stopwords from nltk
#from nltk.corpus import stopwords
#stop_words = stopwords.words('english')
#print(stopwords.words('english'))
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", "won't", 'wouldn', "wouldn't"]

In [432]:
def preprocess1(text):
    result = []
    split=text.split()
 
    for i in split:
        if i not in stop_words:
            result.append(i)
            
    return result

In [433]:
# Apply the function to the dataframe
df['clean'] = df['Statement'].apply(preprocess1)

In [436]:
# Show cleaned statment
df['clean'][8]

['However,',
 'took',
 '$19.5',
 'million',
 'Oregon',
 'Lottery',
 'funds',
 'Port',
 'Newport',
 'eventually',
 'land',
 'new',
 'NOAA',
 'Marine',
 'Operations',
 'Center-Pacific.']

In [367]:
df['Statement'][8]

'However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations Center-Pacific.'

In [437]:
df

,Statement,label,clean
0,Says the Annies List political group supports ...,0,"[Says, Annies, List, political, group, support..."
1,When did the decline of coal start? It started...,1,"[When, decline, coal, start?, It, started, nat..."
2,"Hillary Clinton agrees with John McCain ""by vo...",1,"[Hillary, Clinton, agrees, John, McCain, ""by, ..."
3,Health care reform legislation is likely to ma...,0,"[Health, care, reform, legislation, likely, ma..."
4,The economic turnaround started at the end of ...,1,"[The, economic, turnaround, started, end, term.]"
...,...,...,...
10235,There are a larger number of shark attacks in ...,1,"[There, larger, number, shark, attacks, Florid..."
10236,Democrats have now become the party of the [At...,1,"[Democrats, become, party, [Atlanta], metro, a..."
10237,Says an alternative to Social Security that op...,1,"[Says, alternative, Social, Security, operates..."
10238,On lifting the U.S. Cuban embargo and allowing...,0,"[On, lifting, U.S., Cuban, embargo, allowing, ..."


In [438]:
# Obtain the total words present in the dataset
list_of_words = []
for i in df.clean:
    for j in i:
        list_of_words.append(j)

In [439]:
len(list_of_words)

120981

In [440]:
# Obtain the total number of unique words
total_words = len(list(set(list_of_words)))
total_words

21537

In [441]:
# join the words into a string
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

In [442]:
df['clean_joined'][0]

'Says Annies List political group supports third-trimester abortions demand.'

In [443]:
df

,Statement,label,clean,clean_joined
0,Says the Annies List political group supports ...,0,"[Says, Annies, List, political, group, support...",Says Annies List political group supports thir...
1,When did the decline of coal start? It started...,1,"[When, decline, coal, start?, It, started, nat...",When decline coal start? It started natural ga...
2,"Hillary Clinton agrees with John McCain ""by vo...",1,"[Hillary, Clinton, agrees, John, McCain, ""by, ...","Hillary Clinton agrees John McCain ""by voting ..."
3,Health care reform legislation is likely to ma...,0,"[Health, care, reform, legislation, likely, ma...",Health care reform legislation likely mandate ...
4,The economic turnaround started at the end of ...,1,"[The, economic, turnaround, started, end, term.]",The economic turnaround started end term.
...,...,...,...,...
10235,There are a larger number of shark attacks in ...,1,"[There, larger, number, shark, attacks, Florid...",There larger number shark attacks Florida case...
10236,Democrats have now become the party of the [At...,1,"[Democrats, become, party, [Atlanta], metro, a...",Democrats become party [Atlanta] metro area bl...
10237,Says an alternative to Social Security that op...,1,"[Says, alternative, Social, Security, operates...",Says alternative Social Security operates Galv...
10238,On lifting the U.S. Cuban embargo and allowing...,0,"[On, lifting, U.S., Cuban, embargo, allowing, ...",On lifting U.S. Cuban embargo allowing travel ...


In [444]:
# length of maximum document will be needed to create word embeddings 
maxlen = -1
for doc in df.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is =", maxlen)

The maximum number of words in any document is = 433


## Step5 VISUALIZE CLEANED UP DATASET

In [119]:
!pip install WordCloud
import matplotlib.pyplot as plt
#import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
plt.figure(figsize = (20,20)) 
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop_words).generate(" ".join(df[df.label == 1].clean_joined))
plt.imshow(wc, interpolation = 'bilinear')

## Step 6 PREPARE THE DATA BY PERFORMING TOKENIZATION AND PADDING

In [376]:
import numpy as np

In [445]:
# split data into test and train 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.clean_joined, df.label, test_size = 0.2)

In [378]:
from nltk import word_tokenize

In [379]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [380]:
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer

In [381]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [448]:
# Create a tokenizer to tokenize the words and create sequences of tokenized words
tokenizer = Tokenizer(num_words = total_words)
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)


In [7]:
print("The encoding for document\n",df.clean_joined[0],"\n is : ",train_sequences[0])

AttributeError: 'DataFrame' object has no attribute 'clean_joined'

In [450]:
maxlen

433

In [451]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
padded_train = pad_sequences(train_sequences,maxlen = 20, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 20, padding = 'post',truncating = 'post') 

In [452]:
for i,doc in enumerate(x_train[:2]):
     print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  Says actions Wisconsin commerce secretary brought state 84,000 Wisconsin jobs today.
The padded encoding for document 2  is :  The federal limousine fleet increased 42 percent since Barack Obama took office.


In [453]:
for i,doc in enumerate(padded_train[:2]):
     print("The padded encoding for document",i+1," is : ",doc)

The padded encoding for document 1  is :  [ 392  101  606  388  268  563  268  132 6662  137 1086   62 1286  989
 5577    0    0    0    0    0]
The padded encoding for document 2  is :  [ 227  533   66 1675  739  367   45 1468   38    0    0    0    0    0
    0    0    0    0    0    0]


##  Step9: BUILD AND TRAIN THE MODEL

In [454]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

In [455]:
# Sequential Model
model = Sequential()


In [456]:
# embeddidng layer
model.add(Embedding(total_words, output_dim = 128))
# model.add(Embedding(total_words, output_dim = 240))



In [457]:
# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))

In [458]:
# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 128)         2756736   
                                                                 
 bidirectional_6 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_12 (Dense)            (None, 128)               32896     
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                                 
Total params: 3,052,929
Trainable params: 3,052,929
Non-trainable params: 0
_________________________________________________________________


In [459]:
total_words

21537

In [460]:
y_train = np.asarray(y_train)

In [461]:
# train the model
model.fit(padded_train, y_train, batch_size = 64, validation_split = 0.2, epochs = 4)

Epoch 1/4
103/103 [==============================] - 7s 48ms/step - loss: 0.6715 - acc: 0.5858 - val_loss: 0.6636 - val_acc: 0.6028
Epoch 2/4
103/103 [==============================] - 5s 44ms/step - loss: 0.5468 - acc: 0.7334 - val_loss: 0.7156 - val_acc: 0.5686
Epoch 3/4
103/103 [==============================] - 6s 55ms/step - loss: 0.3538 - acc: 0.8509 - val_loss: 0.8850 - val_acc: 0.5564
Epoch 4/4
103/103 [==============================] - 4s 43ms/step - loss: 0.2101 - acc: 0.9188 - val_loss: 1.2828 - val_acc: 0.5577


## Step10: ASSESS TRAINED MODEL PERFORMANCE

In [462]:
# make prediction
pred = model.predict(padded_test)

64/64 [==============================] - 1s 8ms/step


In [463]:
# if the predicted value is >0.5 it is real else it is fake
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [464]:
# getting the accuracy
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)

Model Accuracy :  0.556640625


## Step11 scoring new data

In [708]:
#df_score = pd.read_csv("/Users/magzhang/Documents/Book1.csv", header=None,names=['Statement'])
df_score_raw = pd.read_csv("/Users/magzhang/Downloads/Text_Grid_view.csv")

In [709]:
df_score_raw

,Question,User_Email,User_rating,correct,PostID
0,rayz1459@gmail.com : is peanuts good for you,rayz1459@gmail.com,True,NaN,1
1,rayz1459@gmail.com : is peanuts good for you,xyz@gmail.com,False,NaN,1
2,rayz1459@gmail.com : is peanuts good for you,abc@gmail.com,True,NaN,1
3,magzhang@linkedin.com: Is Xi a dictator?,cde@gmail.com,False,NaN,2
4,magzhang@linkedin.com: Is Xi a dictator?,xyz@gmail.com,True,NaN,2
5,magzhang@linkedin.com: Is Xi a dictator?,abc@gmail.com,True,NaN,2


In [713]:
df_score_raw[['user', 'Statement']] = df_score_raw['Question'].str.split(pat = ':', expand = True)

df_score= df_score_raw.drop_duplicates(subset=['Statement', 'PostID']).loc[: ,['Statement', 'PostID']]


In [714]:
df_score

,Statement,PostID
0,is peanuts good for you,1
3,Is Xi a dictator?,2


In [610]:
df_score_raw

,Question,User_Email,User_rating,correct,PostID,user,Statement
0,rayz1459@gmail.com : is peanuts good for you,rayz1459@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you
1,rayz1459@gmail.com : is peanuts good for you,xyz@gmail.com,False,NaN,1,rayz1459@gmail.com,is peanuts good for you
2,rayz1459@gmail.com : is peanuts good for you,abc@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you
3,magzhang@linkedin.com: Is Xi a dictator?,cde@gmail.com,False,NaN,2,magzhang@linkedin.com,Is Xi a dictator?
4,magzhang@linkedin.com: Is Xi a dictator?,xyz@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?
5,magzhang@linkedin.com: Is Xi a dictator?,abc@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?


In [712]:
df_score_raw.dtypes


Question        object
User_Email      object
User_rating       bool
correct        float64
PostID           int64
user            object
Statement       object
dtype: object

In [576]:
# Remove stopwords and remove words with 2 or less characters
def preprocess1(text):
    result = []
    split=text.split()
 
    for i in split:
        if i not in stop_words:
            result.append(i)
            
    return result

In [577]:
# Apply the function to the dataframe
df_score['clean'] = df_score['Statement'].apply(preprocess1)

In [578]:
df_score

,Statement,PostID,clean
0,is peanuts good for you,1,"[peanuts, good]"
3,Is Xi a dictator?,2,"[Is, Xi, dictator?]"


In [579]:
# join the words into a string
df_score['clean_joined'] = df_score['clean'].apply(lambda x: " ".join(x))

In [580]:
df_score

,Statement,PostID,clean,clean_joined
0,is peanuts good for you,1,"[peanuts, good]",peanuts good
3,Is Xi a dictator?,2,"[Is, Xi, dictator?]",Is Xi dictator?


In [581]:
x_score=df_score['clean_joined']

In [582]:
x_score

0       peanuts good
3    Is Xi dictator?
Name: clean_joined, dtype: object

In [583]:
# Create a tokenizer to tokenize the words and create sequences of tokenized words
#tokenizer = Tokenizer(num_words = total_words)
#tokenizer.fit_on_texts(x_train)
#train_sequences = tokenizer.texts_to_sequences(x_train)
score_sequences = tokenizer.texts_to_sequences(x_score)


In [584]:
# Add padding can either be maxlen = 4406 or smaller number maxlen = 40 seems to work well based on results
#padded_train = pad_sequences(train_sequences,maxlen = 286, padding = 'post', truncating = 'post')
padded_score = pad_sequences(score_sequences,maxlen = 20, padding = 'post',truncating = 'post') 

In [585]:
score_sequences

[[617], [432, 5570]]

In [586]:
padded_score

array([[ 617,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [ 432, 5570,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [587]:
# make prediction
pred = model.predict(padded_score)

1/1 [==============================] - 0s 39ms/step


In [588]:
pred

array([[0.7308007],
       [0.526001 ]], dtype=float32)

In [589]:
df_score

,Statement,PostID,clean,clean_joined
0,is peanuts good for you,1,"[peanuts, good]",peanuts good
3,Is Xi a dictator?,2,"[Is, Xi, dictator?]",Is Xi dictator?


In [716]:
df_score['pred_score'] = pred

In [717]:
df_score

,Statement,PostID,pred_score
0,is peanuts good for you,1,0.730801
3,Is Xi a dictator?,2,0.526001


## Step12 Calculate the User Credibility Rating

In [775]:
df1=df_score_raw.merge(df_score,left_on='PostID', right_on='PostID')


In [776]:
df1['score_updated'] = df1[['correct', 'pred_score']].bfill(axis=1).iloc[:, 0]

In [777]:
df1

,Question,User_Email,User_rating,correct,PostID,user,Statement_x,Statement_y,pred_score,score_updated
0,rayz1459@gmail.com : is peanuts good for you,rayz1459@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,is peanuts good for you,0.730801,0.730801
1,rayz1459@gmail.com : is peanuts good for you,xyz@gmail.com,False,NaN,1,rayz1459@gmail.com,is peanuts good for you,is peanuts good for you,0.730801,0.730801
2,rayz1459@gmail.com : is peanuts good for you,abc@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,is peanuts good for you,0.730801,0.730801
3,magzhang@linkedin.com: Is Xi a dictator?,cde@gmail.com,False,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,Is Xi a dictator?,0.526001,0.526001
4,magzhang@linkedin.com: Is Xi a dictator?,xyz@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,Is Xi a dictator?,0.526001,0.526001
5,magzhang@linkedin.com: Is Xi a dictator?,abc@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,Is Xi a dictator?,0.526001,0.526001


In [778]:
#This step can be improved by weighting user credibility by pred_score
df2=df1.copy(deep=True)
df2['credibility'] = np.where((df1.score_updated>=0.5) & (df1.User_rating==True),1 ,
                     np.where((df1.score_updated>=0.5) & (df1.User_rating==False), 0,        
                     np.where((df1.score_updated<0.5) & (df1.User_rating==False), 1,         
                     np.where((df1.score_updated<0.5) & (df1.User_rating==True), 0,0))))

In [779]:


df_user=df2.groupby('User_Email').credibility.agg(['count','sum'])

In [780]:
df_user['credibility']=df_user['sum']/df_user['count']

In [781]:
df_user

,count,sum,credibility
User_Email,,,
abc@gmail.com,2,2,1.0
cde@gmail.com,1,0,0.0
rayz1459@gmail.com,1,1,1.0
xyz@gmail.com,2,1,0.5


In [798]:
df1

,Question,User_Email,User_rating,correct,PostID,user,Statement_x,Statement_y,pred_score,score_updated
0,rayz1459@gmail.com : is peanuts good for you,rayz1459@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,is peanuts good for you,0.730801,0.730801
1,rayz1459@gmail.com : is peanuts good for you,xyz@gmail.com,False,NaN,1,rayz1459@gmail.com,is peanuts good for you,is peanuts good for you,0.730801,0.730801
2,rayz1459@gmail.com : is peanuts good for you,abc@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,is peanuts good for you,0.730801,0.730801
3,magzhang@linkedin.com: Is Xi a dictator?,cde@gmail.com,False,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,Is Xi a dictator?,0.526001,0.526001
4,magzhang@linkedin.com: Is Xi a dictator?,xyz@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,Is Xi a dictator?,0.526001,0.526001
5,magzhang@linkedin.com: Is Xi a dictator?,abc@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,Is Xi a dictator?,0.526001,0.526001


## Step13 Calibrate final score with user rating

In [804]:
df3=df1.merge(df_user,left_on='User_Email',right_on='User_Email',how='left')

df3.credibility.fillna(value=0.5, inplace=True)

In [808]:
df3.drop(['Question','Statement_y','count','sum'],axis=1, inplace=True)

In [812]:
df3.rename(columns={'Statement_x':'statement'}, inplace=True)

In [813]:
df3

,User_Email,User_rating,correct,PostID,user,statement,pred_score,score_updated,credibility
0,rayz1459@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,0.730801,0.730801,1.0
1,xyz@gmail.com,False,NaN,1,rayz1459@gmail.com,is peanuts good for you,0.730801,0.730801,0.5
2,abc@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,0.730801,0.730801,1.0
3,cde@gmail.com,False,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,0.526001,0.526001,0.0
4,xyz@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,0.526001,0.526001,0.5
5,abc@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,0.526001,0.526001,1.0


In [815]:
df3['user_rating1']=np.where(df3['User_rating'] == True,1,-1)

In [816]:
df3['user_w_rating']=df3.user_rating1*df3.credibility

In [2]:
#df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})

df4=df3.groupby('PostID').agg({'user':'count','user_w_rating':'sum','score_updated':'mean'})

NameError: name 'df3' is not defined

In [840]:
df3

,User_Email,User_rating,correct,PostID,user,statement,pred_score,score_updated,credibility,user_rating1,user_w_rating
0,rayz1459@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,0.730801,0.730801,1.0,1,1.0
1,xyz@gmail.com,False,NaN,1,rayz1459@gmail.com,is peanuts good for you,0.730801,0.730801,0.5,-1,-0.5
2,abc@gmail.com,True,NaN,1,rayz1459@gmail.com,is peanuts good for you,0.730801,0.730801,1.0,1,1.0
3,cde@gmail.com,False,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,0.526001,0.526001,0.0,-1,-0.0
4,xyz@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,0.526001,0.526001,0.5,1,0.5
5,abc@gmail.com,True,NaN,2,magzhang@linkedin.com,Is Xi a dictator?,0.526001,0.526001,1.0,1,1.0


In [842]:
df4.rename(columns={'user':'n_reply','user_w_rating':'user_sum_score'}, inplace=True)

In [1]:
df4

NameError: name 'df4' is not defined

In [844]:
df4['final_score']=df4.score_updated*0.75+0.25*(df4.user_sum_score/df4.n_reply/2+0.5)

In [ ]:
#df4 is the final output table